## CASO PRACTICO

importar la DataBase de PgAdmin4, hacer las consultas sql y graficar

In [43]:
import pandas as pd
from sqlalchemy import create_engine 

In [44]:
#conexion a la base de datos de postgresql
username = 'postgres'
password = '7138'
host = 'localhost'
port = '5432'
database = 'DataBase'

engine =create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

In [45]:
#Leer o cargar las 3 tablas, ventas, clientes, productos
df_ventas=pd.read_sql('SELECT * FROM ventas',con=engine)
df_ventas

,id_venta,id_cliente,id_producto,cantidad,fecha,precio_unitario,total_venta
0,1,1,1,2,2025-08-21,25.50,51.00
1,2,2,3,1,2025-08-21,32.75,32.75
2,3,3,4,5,2025-08-21,28.40,142.00
3,4,4,2,3,2025-08-21,18.00,54.00
4,5,5,5,4,2025-08-21,22.90,91.60
5,6,1,1,2,2025-08-01,15.50,31.00
6,7,2,2,1,2025-08-01,22.00,22.00
7,8,3,3,3,2025-08-02,18.75,56.25
8,9,4,1,1,2025-08-02,15.50,15.50
9,10,5,4,2,2025-08-03,25.00,50.00


In [46]:
df_clientes=pd.read_sql('SELECT * FROM clientes', con=engine)
df_clientes

,id_cliente,nombre,email,telefono
0,1,Juan Pérez,juan.perez@example.com,987654321
1,2,María López,maria.lopez@example.com,976543210
2,3,Carlos Ramírez,carlos.ramirez@example.com,965432187
3,4,Ana Torres,ana.torres@example.com,954321876
4,5,Luis Fernández,luis.fernandez@example.com,943218765


In [47]:
df_productos=pd.read_sql('SELECT * FROM productos', con=engine)
df_productos

,id_producto,nombre,categoria,precio
0,1,Miel de abeja orgánica,Miel,25.50
1,2,Miel multifloral,Miel,18.00
2,3,Cacao en grano premium,Cacao,32.75
3,4,Café arábica molido,Café,28.40
4,5,Café robusta tostado,Café,22.90


In [48]:
#comando flexible para poder leer las tablas (prodcutos)
df_productos=pd.read_sql('productos',con=engine)
df_productos

,id_producto,nombre,categoria,precio
0,1,Miel de abeja orgánica,Miel,25.50
1,2,Miel multifloral,Miel,18.00
2,3,Cacao en grano premium,Cacao,32.75
3,4,Café arábica molido,Café,28.40
4,5,Café robusta tostado,Café,22.90


## Uniones de tabla - Cruces de tabla

existen 2 metodos para hacer los cruces:

- join en sql
- merge en pandas

con pd.merge sí es conceptualmente lo mismo que un JOIN en SQL, pero hay matices que conviene tener claros.

Diferencias:

* JOIN en SQL → el trabajo lo hace Postgres antes de traerte los datos.
* merge en Pandas → primero traes cada tabla a memoria en Python y después haces la unión. 

Escalabilidad:

* SQL es más eficiente si tus tablas tienen millones de registros (Postgres optimiza los joins).

* Pandas es cómodo si ya tienes las tablas en memoria y vas a hacer análisis inmediato.

Sintaxis:

* pd.merge soporta los mismos tipos de JOIN que SQL (left, right, inner, outer).

* Necesitas que las columnas de unión tengan el mismo nombre, o de lo contrario debes usar left_on y right_on.

In [49]:
#Mostrar el total de ventas por clientes usando "join"

query = """
    SELECT 
        c.id_cliente,  
        c.nombre as cliente,
        SUM(v.total_venta) AS total_gastado,
        COUNT(v.id_venta) AS numero_compras
    FROM ventas v
    JOIN clientes c ON v.id_cliente = c.id_cliente
    GROUP BY c.id_cliente, c.nombre
    ORDER BY total_gastado DESC
"""
df_ventas_clientes = pd.read_sql(query,con=engine)
df_ventas_clientes

,id_cliente,cliente,total_gastado,numero_compras
0,3,Carlos Ramírez,315.75,5
1,5,Luis Fernández,298.10,5
2,4,Ana Torres,281.00,5
3,2,María López,243.50,5
4,1,Juan Pérez,182.75,5


In [50]:
#Mostrar el total de ventas por clientes usando "merge"

df_ventas_clientes = pd.merge(
    df_ventas,
    df_clientes,
    on="id_cliente",
    how="left"   # equivale a LEFT JOIN
)

df_resumen = (
    df_ventas_clientes
    .groupby(["id_cliente", "nombre"], as_index=False)
    .agg( #la funcion aggregate se usa siempre despues de un groupby
        total_gastado=("total_venta", "sum"),
        numero_compras=("id_venta", "count")
    )
    .sort_values("total_gastado", ascending=False)
)
df_resumen



,id_cliente,nombre,total_gastado,numero_compras
2,3,Carlos Ramírez,315.75,5
4,5,Luis Fernández,298.10,5
3,4,Ana Torres,281.00,5
1,2,María López,243.50,5
0,1,Juan Pérez,182.75,5


In [51]:
df_ventas_clientes=pd.merge (
    df_ventas,
    df_clientes,
    on='id_cliente',
    how='left'
)

df_resumen = (
    df_ventas_clientes
    .groupby (['id_cliente','nombre'], as_index=False)
    .agg ( #aplica funciones de agregación a las columnas seleccionadas.
        total_gastado=('total_venta','sum'),
        numero_compra=('id_venta','count') 
    )
    .sort_values('total_gastado', ascending=False) #Es como hacer un ORDER BY en SQL o un ordenar de mayor a menor en Excel.
)
df_resumen

,id_cliente,nombre,total_gastado,numero_compra
2,3,Carlos Ramírez,315.75,5
4,5,Luis Fernández,298.10,5
3,4,Ana Torres,281.00,5
1,2,María López,243.50,5
0,1,Juan Pérez,182.75,5


In [52]:
#Mostrar el producto mas vendido

query2= ("""
    SELECT 
        p.id_producto,
        p.nombre AS producto,
        COUNT (v.id_venta) AS cantidad_vendida
    FROM productos p
    JOIN ventas v ON p.id_producto = v.id_producto
    GROUP BY p.id_producto, p.nombre
    ORDER BY cantidad_vendida DESC
    LIMIT 1;
""")
df_producto_mas_vendido=pd.read_sql(query2,con=engine)
df_producto_mas_vendido

,id_producto,producto,cantidad_vendida
0,2,Miel multifloral,6


In [53]:
# unir ventas con productos
df_ventas_productos = pd.merge(
    df_ventas,
    df_productos,
    on="id_producto",
    how="left"
)

# agrupar y ordenar
df_producto_mas_vendido = (
    df_ventas_productos
    .groupby(["id_producto", "nombre"], as_index=False)
    .agg(cantidad_vendida=("id_venta", "count"))
    .sort_values("cantidad_vendida", ascending=False)
    .head(1)
)

print(df_producto_mas_vendido)


   id_producto            nombre  cantidad_vendida
1            2  Miel multifloral                 6


In [55]:
producto_mas_vendido=pd.merge(
    df_ventas,
    df_productos,
    on='id_producto',
    how='left'
)

resultado2=(
    producto_mas_vendido
    .groupby(['id_producto','nombre'], as_index=False)
    .agg(
        cantidad_vendida=('id_venta','count')
    )
    .sort_values('cantidad_vendida', ascending=False)
    .head(2)
)
resultado2

,id_producto,nombre,cantidad_vendida
1,2,Miel multifloral,6
0,1,Miel de abeja orgánica,5


In [60]:
#Ventas totales por categoria

query3 = ("""
    SELECT 
        p.id_producto,
        p.categoria AS categoria,
        SUM (v.total_venta) AS total_vendido,
        COUNT (v.id_venta) AS cantidad_ventas
    FROM productos p
    JOIN ventas v ON p.id_producto = v.id_producto
    GROUP BY p.id_producto, p.categoria
    ORDER BY total_vendido DESC
""")
ventas_categoria=pd.read_sql(query3,con=engine)
ventas_categoria


,id_producto,categoria,total_vendido,cantidad_ventas
0,5,Café,391.6,5
1,4,Café,292.0,4
2,2,Miel,252.0,6
3,1,Miel,221.5,5
4,3,Cacao,164.0,5


In [66]:
#total ventas por categoria usando merge

categoria_ventas=pd.merge(
    df_productos,
    df_ventas,
    on='id_producto',
    how='left'
)

result3=(
    categoria_ventas
    .groupby(['id_producto','categoria'], as_index=False)
    .agg(
        total_vendido=('total_venta','sum'),
        cantidad_ventas=('id_venta','count')
    )
    .sort_values('total_vendido',ascending=False)
)
result3

,id_producto,categoria,total_vendido,cantidad_ventas
4,5,Café,391.6,5
3,4,Café,292.0,4
1,2,Miel,252.0,6
0,1,Miel,221.5,5
2,3,Cacao,164.0,5
